# 5. Basin Attribute Extraction 

```{figure} img/nalcms_VI.gif
---
width: 600px
---
North American Land Change Monitoring System {cite}`latifovic2010north` rasters covering Vancouver Island for 2010, 2015, and 2020. 
```

The final step is to capture geospatial information describing the soil, land cover, and terrain of each basin using the polygons we developed in the previous notebook.  First we need to get the data and trim it to the Vancouver Island polygon.

In [1]:
# isolate the GLHYMPS data
import os
import time
from utilities import retrieve_raster
import geopandas as gpd
import numpy as np
import pandas as pd

from numba import jit

from scipy.stats.mstats import gmean


## Download and Clip NALCMS Data

North American Coverage of the NALCMS can be downloaded from [North American Land Change Monitoring System (NALCMS)](http://www.cec.org/north-american-land-change-monitoring-system/).  

Download the files you want to work with from the link above, and use the steps below to crop the dataset to the region of interest (Vancouver Island).

In [48]:
# set the path to the downloaded NALCMS file
nalcms_fpath = # path/to/nalcms_file
nalcms_raster, nalcms_crs, nalcms_affine = retrieve_raster(nalcms_fpath)

SyntaxError: invalid syntax (3097683402.py, line 2)

In [40]:
# set the path to the polygon mask for clipping the NALCMS raster
base_dir = os.path.dirname(os.getcwd())
year = 2015
region = 'Vancouver_Island'
polygon_path = os.path.join(os.getcwd(), f'data/region_polygons/{region}.geojson')
region_polygon = gpd.read_file(polygon_path)

Set the DEM path which was created in notebook 2.  Use the original DEM, not the (pit/depression) filled DEM.

In [41]:
# this should be the same path as Notebook 2
dem_dir = os.path.join(base_dir, 'notebooks/data/DEM/')
dem_fpath = os.path.join(dem_dir, f'{region}_3005.tif')

In [36]:
# create a folder for the output geospatial layers
output_folder = os.path.join(os.getcwd(), 'data/geospatial_layers')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

We need the polygon mask to have the same CRS as the data source we are clipping.

In [43]:
# region_polygon = region_polygon.to_crs(nalcms_crs)
if not region_polygon.crs == nalcms_crs:
    region_polygon = region_polygon.to_crs(nalcms_crs)


In [10]:
output_dem_path = os.path.join(output_folder, f'NALCMS_{year}_{region}.tif')
command = f'gdalwarp -s_srs epsg:{crs} -cutline {polygon_path} -cl Vancouver_Island -crop_to_cutline -multi -of gtiff {nalcms_fpath} {output_dem_path} -wo NUM_THREADS=ALL_CPUS'
if not os.path.exists(output_dem_path):
    os.system(command)

## Download and Clip GLHYMPS Data

```{figure} img/perm_porosity.png
---
width: 600px
---
The GLobal HYdrogeology MaPS (GLHYMPS) {cite}`SP2_TTJNIU_2018` is global coverage of permeability and porosity in vector format.  

Download the file from [here](https://aquaknow.jrc.ec.europa.eu/en/content/global-hydrogeology-maps-glhymps-permeability-and-porosity), and use the steps below to clip the data to the region of interest.  

Note that this file is large, and it's necessary to use the `mask` feature when opening the file using geopandas. Expect the opening and masking to take several minutes.

In [47]:
# glhymps is in EPSG 4326, ensure the polygon is the same CRS
region_polygon = region_polygon.to_crs(4326)    

In [ ]:
glhymps_path = # path to glhymps file
gldf = gpd.read_file(glhymps_path, mask=region_polygon)

In [12]:
glhymps_output_path = os.path.join(output_folder, f'GLHYMPS_{region}.gpkg')
gldf.to_file(glhymps_output_path)

## Direct Attribute Retrieval

Here we load the basin polygon batch files produced in the last notebook, and iterate through polygons to extract attributes as we go.  This method is not the most performant, but the details of the process are hopefully clear.

In [25]:
basins_folder = os.path.join(os.getcwd(), 'data/basins/')
batches = os.listdir(basins_folder)
file = batches[0]
basins_df = gpd.read_file(os.path.join(basins_folder, file))


**Table: Set of metadata and catchment attributes in the BCUB database derived from USGS 3DEP (DEM), NALCMS (land cover), and GLHYMPS (soil) datasets.**

| **Group**  | **Description (BCUB label)** | **Aggregation** | **Units**       |
|------------|------------------------------|-----------------|-----------------|
| Metadata   | Pour point (geom)            | -               | decimal deg.$^1$|
|            | Basin centroid point (centroid)| -             | decimal deg.    |
|            | Land Cover Flag (lulc_check) | -               | binary (0/1)    |
|------------|------------------------------|-----------------|-----------------|
| Terrain    | Drainage Area (drainage_area_km2)| at pour point | $km^2$          |
|            | Elevation (elevation_m)       | spatial mean    | $m$ above sea level|
|            | Terrain Slope (slope_deg)     | spatial mean    | $^\circ$ (degrees)|
|            | Terrain Aspect (aspect_deg)   | circular mean$^2$| $^\circ$ (degrees)|
|------------|------------------------------|-----------------|-----------------|
| Land Cover$^3$ | Cropland (land_use_crops_frac_<year>) | -     |                   |
|            | Forest (land_use_forest_frac_<year>)   | -      |                   |
|            | Grassland (land_grass_forest_frac_<year>)| -    |                   |
|            | Shrubs (land_use_shrubs_frac_<year>)     | spatial mean| $\%$ cover    |
|            | Snow & Ice (land_use_snow_ice_frac_<year>)| -  |                   |
|            | Urban (land_use_urban_frac_<year>)       | -      |                   |
|            | Water (land_use_water_frac_<year>)       | -      |                   |
|            | Wetland (land_use_wetland_frac_<year>)   | -      |                   |
|------------|------------------------------|-----------------|-----------------|
| Soil       | Permeability (permeability_logk_m2)      | geometric mean | $m^2$        |
|            | Porosity (porosity_frac)      | spatial mean    | $\%$ cover     |

**Notes**:
1.  Geometries are formatted in the WSG84 coordinate reference system.
2.  Spatial aspect is expressed in degrees counter-clockwise from the east direction.
3.  The <year> suffix specifies the land cover dataset (2010, 2015, or 2020).



## Land Cover Data

Below we use LULC to refer to "land use land cover".

In [14]:
from attribute_functions import clip_raster_to_basin

In [15]:
def check_lulc_sum(data):
    """
    Check if the sum of pct. land cover sums to 1.
    Return value is 1 - sum to correspond with 
    a more intuitive boolean flag, 
    i.e. data quality flags are 1 if the flag is raised,
    0 of no flag.
    """
    checksum = sum(list(data.values())) 
    lulc_check = 1-checksum
    if abs(lulc_check) >= 0.05:
        print(f'   ...checksum failed: {checksum:.3f}')   
    return lulc_check


def recategorize_lulc(data):    
    forest = ('Land_Use_Forest_frac', [1, 2, 3, 4, 5, 6])
    shrub = ('Land_Use_Shrubs_frac', [7, 8, 11])
    grass = ('Land_Use_Grass_frac', [9, 10, 12, 13, 16])
    wetland = ('Land_Use_Wetland_frac', [14])
    crop = ('Land_Use_Crops_frac', [15])
    urban = ('Land_Use_Urban_frac', [17])
    water = ('Land_Use_Water_frac', [18])
    snow_ice = ('Land_Use_Snow_Ice_frac', [19])
    lulc_dict = {}
    for label, p in [forest, shrub, grass, wetland, crop, urban, water, snow_ice]:
        prop_vals = round(sum([data[e] if e in data.keys() else 0.0 for e in p]), 2)
        lulc_dict[label] = prop_vals
    return lulc_dict
    

def get_value_proportions(data):
    # create a dictionary of land cover values by coverage proportion
    # assuming raster pixels are equally sized, we can keep the
    # raster in geographic coordinates and just count pixel ratios
    all_vals = data.data.flatten()
    vals = all_vals[~np.isnan(all_vals)]
    n_pts = len(vals)
    unique, counts = np.unique(vals, return_counts=True)    
    prop_dict = {k: 1.0*v/n_pts for k, v in zip(unique, counts)}

    if 15 in prop_dict.keys():        
        if prop_dict[15] > 0.01:
            print(prop_dict)
            
    prop_dict = recategorize_lulc(prop_dict)
    return prop_dict    


def process_lulc(basin_geom, nalcms_raster):
    # polygon = basin_polygon.to_crs(nalcms_crs)
    # assert polygon.crs == nalcms.rio.crs
    basin_id = basin_geom['VALUE'].values[0]
    raster_loaded, lu_raster_clipped = clip_raster_to_basin(basin_geom, nalcms_raster)
    # checksum verifies proportions sum to 1
    prop_dict = get_value_proportions(lu_raster_clipped)
    lulc_check = check_lulc_sum(prop_dict)
    prop_dict['lulc_check'] = lulc_check
    return pd.DataFrame(prop_dict, index=[basin_id])

In [16]:
def check_and_repair_geometries(in_feature):

    # avoid changing original geodf
    in_feature = in_feature.copy(deep=True)    
        
    # drop any missing geometries
    in_feature = in_feature[~(in_feature.is_empty)]
    
    # Repair broken geometries
    for index, row in in_feature.iterrows(): # Looping over all polygons
        if row['geometry'].is_valid:
            next
        else:
            fix = make_valid(row['geometry'])
            try:
                in_feature.loc[[index],'geometry'] =  fix # issue with Poly > Multipolygon
            except ValueError:
                in_feature.loc[[index],'geometry'] =  in_feature.loc[[index], 'geometry'].buffer(0)
    return in_feature


In [17]:
def process_basin_elevation(clipped_raster):
    # evaluate masked raster data
    values = clipped_raster.data.flatten()
    mean_val = np.nanmean(values)
    median_val = np.nanmedian(values)
    min_val = np.nanmin(values)
    max_val = np.nanmax(values)
    return mean_val, median_val, min_val, max_val

In [18]:
def get_soil_properties(merged, col):
    # dissolve polygons by unique parameter values
    geometries = check_and_repair_geometries(merged)

    df = geometries[[col, 'geometry']].copy().dissolve(by=col, aggfunc='first')
    df[col] = df.index.values
    # re-sum all shape areas
    df['Shape_Area'] = df.geometry.area
    # calculuate area fractions of each unique parameter value
    df['area_frac'] = df['Shape_Area'] / df['Shape_Area'].sum()
    # check that the total area fraction = 1
    total = round(df['area_frac'].sum(), 1)
    sum_check = total == 1.0
    if not sum_check:
        print(f'    Area proportions do not sum to 1: {total:.2f}')
        if np.isnan(total):
            return np.nan
        elif total < 0.9:
            return np.nan
    
    # area_weighted_vals = df['area_frac'] * df[col]
    if 'Permeability' in col:
        # calculate geometric mean
        # here we change the sign (all permeability values are negative)
        # and add it back at the end by multiplying by -1 
        # otherwise the function tries to take the log of negative values
        return gmean(np.abs(df[col]), weights=df['area_frac']) * -1
    else:
        # calculate area-weighted arithmetic mean
        return (df['area_frac'] * df[col]).sum()
    

def process_glhymps(basin_geom, fpath):
    # import soil layer with polygon mask (both in 4326)
    basin_geom = basin_geom.to_crs(4326)
    # returns INTERSECTION
    gdf = gpd.read_file(fpath, mask=basin_geom)
    # now clip precisely to the basin polygon bounds
    merged = gpd.clip(gdf, mask=basin_geom)
    # now reproject to minimize spatial distortion
    merged = merged.to_crs(3005)
    return merged


In [49]:

@jit(nopython=True)
def process_slope_and_aspect(E, el_px, resolution, shape):
    # resolution = E.rio.resolution()
    # shape = E.rio.shape
    # note, distances are not meaningful in EPSG 4326
    # note, we can either do a costly reprojection of the dem
    # or just use the approximate resolution of 90x90m
    # dx, dy = 90, 90# resolution
    dx, dy = resolution
    # print(resolution)
    # print(asdfd)
    # dx, dy = 90, 90
    S, A = np.empty_like(E), np.empty_like(E)
    S[:] = np.nan # track slope (in degrees)
    A[:] = np.nan # track aspect (in degrees)
    # tot_p, tot_q = 0, 0
    for i, j in el_px:
        if (i == 0) | (j == 0) | (i == shape[0]) | (j == shape[1]):
            continue
            
        E_w = E[i-1:i+2, j-1:j+2]

        if E_w.shape != (3,3):
            continue

        a = E_w[0,0]
        b = E_w[1,0]
        c = E_w[2,0]
        d = E_w[0,1]
        f = E_w[2,1]
        g = E_w[0,2]
        h = E_w[1,2]
        # skip i and j because they're already used
        k = E_w[2,2]  

        all_vals = np.array([a, b, c, d, f, g, h, k])

        val_check = np.isfinite(all_vals)

        if np.all(val_check):
            p = ((c + 2*f + k) - (a + 2*d + g)) / (8 * abs(dx))
            q = ((c + 2*b + a) - (k + 2*h + g)) / (8 * abs(dy))
            cell_slope = np.sqrt(p*p + q*q)
            S[i, j] = (180 / np.pi) * np.arctan(cell_slope)
            A[i, j] = (180.0 / np.pi) * np.arctan2(q, p)

    return S, A


def calculate_circular_mean_aspect(a):
    """
    From RavenPy:
    https://github.com/CSHS-CWRA/RavenPy/blob/1b167749cdf5984545f8f79ef7d31246418a3b54/ravenpy/utilities/analysis.py#L118
    """
    angles = a[~np.isnan(a)]
    n = len(angles)
    sine_mean = np.divide(np.sum(np.sin(np.radians(angles))), n)
    cosine_mean = np.divide(np.sum(np.cos(np.radians(angles))), n)
    vector_mean = np.arctan2(sine_mean, cosine_mean)
    degrees = np.degrees(vector_mean)
    if degrees < 0:
        return degrees + 360
    else:
        return degrees


def calculate_slope_and_aspect(raster):  
    """Calculate mean basin slope and aspect 
    according to Hill (1981).

    Args:
        clipped_raster (array): dem raster

    Returns:
        slope, aspect: scalar mean values
    """
    # print(raster.data[0])
    # print(raster.rio.crs)
    # print(asfd)
    # wkt = raster.rio.crs.to_wkt()
    # affine = raster.rio.transform()

    resolution = raster.rio.resolution()
    raster_shape = raster[0].shape

#     rdem_clipped = rd.rdarray(
#         raster.data[0], 
#         no_data=raster.rio.nodata, 
#         projection=wkt, 
#     )

#     rdem_clipped.geotransform = affine.to_gdal()
#     rdem_clipped.projection = wkt

    # # ts0 = time.time()
    # use to check slope -- works out to within 1 degree...
    # slope = rd.TerrainAttribute(rdem_clipped, attrib='slope_degrees')
    # aspect_deg = rd.TerrainAttribute(rdem_clipped, attrib='aspect')
    # # ts2 = time.time()

    el_px = np.argwhere(np.isfinite(raster.data[0]))

    # print(el_px[:2])
    # print(rdem_clipped)
    # print(asdfsd)
    S, A = process_slope_and_aspect(raster.data[0], el_px, resolution, raster_shape)

    mean_slope_deg = np.nanmean(S)
    # should be within a hundredth of a degree or so.
    # print(f'my slope: {mean_slope_deg:.4f}, rdem: {np.nanmean(slope):.4f}')
    mean_aspect_deg = calculate_circular_mean_aspect(A)

    return mean_slope_deg, mean_aspect_deg

In [20]:
year = 2010
nalcms_fpath = output_dem_path
nalcms, nalcms_crs, nalcms_affine = retrieve_raster(nalcms_fpath)

region_dem, dem_crs, dem_affine = retrieve_raster(dem_fpath)

In [21]:
# to test, try on a few samples, note how basins_df is sliced in the for .. statement next 
# to run the whole batch, remove the [:n_samples] slice
n_samples = 10

In [22]:
all_basin_data = []
t0 = time.time()
for i, row in basins_df[:n_samples].iterrows():
    basin_data = {}
    basin_data['region'] = region
    basin_data['id'] = row['VALUE']

    basin_polygon = basins_df.iloc[[i]].copy()
    
    clip_ok, clipped_dem = clip_raster_to_basin(basin_polygon, region_dem)
    
    
    land_cover = process_lulc(basin_polygon, nalcms)
    land_cover = land_cover.to_dict('records')[0]
    # print('     lulc complate')
    basin_data.update(land_cover)
    
    soil = process_glhymps(basin_polygon, glhymps_path)
    porosity = get_soil_properties(soil, 'Porosity')
    permeability = get_soil_properties(soil, 'Permeability_no_permafrost')
    basin_data['Permeability_logk_m2'] = round(permeability, 2)
    basin_data['Porosity_frac'] = round(porosity, 5)
    
    slope, aspect = calculate_slope_and_aspect(clipped_dem)
    # print(f'aspect, slope: {aspect:.1f} {slope:.2f} ')
    basin_data['Slope_deg'] = slope
    basin_data['Aspect_deg'] = aspect


    mean_el, median_el, min_el, max_el = process_basin_elevation(clipped_dem)
    basin_data['median_el'] = median_el
    basin_data['mean_el'] = mean_el
    basin_data['max_el'] = max_el
    basin_data['min_el'] = min_el
    
    all_basin_data.append(basin_data)

t1 = time.time()
ut = (t1 - t0) / n_samples

In [28]:
output = pd.DataFrame(all_basin_data)
print(f'Processed {n_samples} basins in {t1-t0:.0f}s ({ut:.2f}s/basin)')
# save the file
output_fname = file.replace('.geojson', '.csv')
output_file = output.to_csv(os.path.join(base_dir, f'notebooks/data/{output_fname}'))

Processed 10 basins in 3s (0.30s/basin)


Median processing time is roughly 1s/basin on a ~2018 Intel i7-8850H CPU @ 2.60GHz.  Processing time is proporti onal to the size of clipped DEM, and using JIT in the `process_slope_and_aspect` function yields ~3X performance improvement.

## References

```{bibliography} 
```